In [1]:
import nltk
import numpy as np
from tqdm.notebook import tqdm_notebook 
import tqdm
import pandas as pd
import os
import math
import warnings
warnings.filterwarnings('ignore')
import argparse
import timeit

from ir_system import IRSystem

In [4]:
df_freqs_poids = pd.read_csv('freq_poids_porter.csv')
df_freqs_poids

,Document,Terme,Frequence,Poid
0,1,18,1,0.565909
1,1,edit,4,1.543491
2,1,dewey,3,1.566545
3,1,decim,2,0.982483
4,1,classif,2,0.586663
...,...,...,...,...
77166,1460,100,1,0.630405
77167,1460,150,1,0.896041
77168,1460,thousand,1,0.674123
77169,1460,new,1,0.283413


In [22]:
df_freqs_poids['Terme'].unique().shape
unique_terms = list(df_freqs_poids['Terme'].unique())
len(unique_terms)

6924

In [20]:
df_freqs_poids.loc[df_freqs_poids['Document'] == 1]
df = pd.DataFrame(columns = unique_terms)
df

,18,edit,dewey,decim,classif,present,studi,histori,first,ddc,...,anglo,monopoli,patente,sell,inventor,claus,morn,supposit,15%,9%


In [26]:
for document in df_freqs_poids['Document'].unique():
    temp = df_freqs_poids.loc[df_freqs_poids['Document'] == document]
    temp = temp.drop(['Frequence'], axis=1)
    df.loc[document] = pd.Series((pd.Series(temp.Poid.values,index=temp.Terme.values).to_dict()))

In [28]:
df.fillna(0, inplace=True)
df

,18,edit,dewey,decim,classif,present,studi,histori,first,ddc,...,anglo,monopoli,patente,sell,inventor,claus,morn,supposit,15%,9%
1,0.565909,1.543491,1.566545,0.982483,0.586663,0.187722,0.175460,0.735847,0.488596,1.233434,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.162865,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140368,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1457,0.000000,0.000000,0.000000,0.000000,0.000000,0.125148,0.000000,0.000000,0.000000,0.000000,...,0.527442,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1458,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.054883,1.054883,1.054883,1.054883,1.054883,1.054883,1.054883,0.000000,0.000000
1459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
df.to_csv('df_poids.csv')

## DBscan

In [30]:
from sklearn.cluster import DBSCAN


In [167]:
model = DBSCAN(eps=5, min_samples=1).fit(df)
labels = np.array(list(model.labels_))
np.unique(labels)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148], dtype=int64)

In [168]:
labels[labels == -1].shape

(0,)

In [169]:
labels[labels == 0].shape

(1312,)

In [170]:
labels[labels == 1].shape

(1,)

### PCA

In [155]:
from sklearn.decomposition import PCA

In [156]:
pca = PCA(n_components=1400)
pca.fit(df)
x_pca = pca.transform(df)
pca_df = pd.DataFrame(x_pca)
pca_df

,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,0.173932,-0.032772,-0.059608,0.018051,-0.107983,0.156326,-0.348169,0.110584,0.103923,-0.224260,...,-0.007944,-0.005509,0.000365,-0.006223,0.007714,-0.000084,-0.004853,0.001684,-0.005853,0.001553
1,0.178540,-0.030328,0.108434,-0.066117,-0.070030,-0.141931,0.014205,-0.122806,-0.153678,0.062255,...,0.079062,-0.039514,-0.032026,-0.047048,0.006927,0.007311,0.069762,0.064180,0.011223,-0.015401
2,0.094648,0.033476,-0.100488,0.079230,-0.115752,-0.069461,-0.042223,0.036401,0.211386,-0.157137,...,-0.007794,-0.001515,0.005503,-0.001600,0.010963,-0.001685,-0.008723,-0.007109,-0.000187,-0.020129
3,0.339146,-0.077031,0.325844,0.010194,-0.098761,-0.117374,-0.041331,-0.124105,-0.168112,0.139289,...,0.001259,-0.008194,0.004848,0.003265,-0.001018,-0.010198,0.000906,-0.005050,-0.006685,-0.000364
4,0.180564,0.001671,-0.003416,0.151790,0.019720,-0.107551,-0.016031,0.045620,-0.047291,0.043795,...,-0.008221,-0.000584,0.000540,0.003320,-0.000272,-0.001368,0.005922,-0.003572,-0.002782,0.000303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.247543,-0.014180,-0.189948,0.065382,-0.078147,-0.069687,0.014062,0.099765,0.021630,-0.014740,...,0.004978,-0.008367,0.018619,-0.006732,-0.004970,-0.014897,0.005070,0.008450,0.016878,0.018937
1456,0.466476,-0.017371,0.192728,0.034109,0.035473,-0.126007,-0.148607,-0.011447,-0.102123,0.090449,...,0.030521,-0.005268,0.007856,0.032510,-0.008139,0.002835,-0.016556,-0.017099,0.023231,-0.004810
1457,0.308791,0.027076,-0.334023,0.071095,-0.086465,-0.053822,-0.008096,0.016213,0.222893,-0.133925,...,0.001933,-0.002583,-0.003035,0.000022,0.003456,0.001786,0.003904,0.000915,0.001509,-0.003565
1458,-0.076540,0.016487,-0.175375,0.104849,-0.248866,0.077568,-0.167513,-0.258513,0.119206,0.142958,...,-0.014750,-0.005775,-0.011875,-0.008764,0.000539,0.004201,-0.010546,0.016745,-0.002685,0.007534


In [157]:
## try to normalize the data
from sklearn import preprocessing

x = pca_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
pca_df_normalized = pd.DataFrame(x_scaled)

In [158]:
pca_df_normalized

,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,0.541526,0.051688,0.359678,0.138311,0.163319,0.083694,0.241397,0.454150,0.375182,0.061486,...,0.471376,0.457532,0.461661,0.420322,0.516412,0.393330,0.446624,0.481992,0.347315,0.413306
1,0.542728,0.051879,0.400105,0.128841,0.167795,0.047259,0.289900,0.418436,0.284447,0.095599,...,0.696575,0.343716,0.363768,0.322993,0.514325,0.411158,0.622697,0.660713,0.387526,0.377076
2,0.520847,0.056865,0.349843,0.145195,0.162402,0.056112,0.282347,0.442799,0.413034,0.069478,...,0.471764,0.470900,0.477190,0.431345,0.525024,0.389472,0.437491,0.456848,0.360658,0.366972
3,0.584617,0.048230,0.452410,0.137427,0.164406,0.050259,0.282466,0.418237,0.279363,0.104771,...,0.495196,0.448546,0.475212,0.442943,0.493263,0.368949,0.460214,0.462738,0.345357,0.409209
4,0.543256,0.054380,0.373197,0.153359,0.178381,0.051459,0.285853,0.444209,0.321920,0.093401,...,0.470659,0.474017,0.462192,0.443073,0.495240,0.390235,0.472049,0.466962,0.354546,0.410634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.560726,0.053141,0.328321,0.143636,0.166838,0.056084,0.289880,0.452495,0.346196,0.086432,...,0.504822,0.447967,0.516831,0.419108,0.482785,0.357623,0.470040,0.501342,0.400842,0.450457
1456,0.617827,0.052892,0.420385,0.140118,0.180239,0.049204,0.268108,0.435477,0.302606,0.098956,...,0.570935,0.458338,0.484301,0.512664,0.474384,0.400368,0.419006,0.428281,0.415803,0.399709
1457,0.576700,0.056365,0.293660,0.144279,0.165857,0.058022,0.286915,0.439709,0.417087,0.072242,...,0.496941,0.467326,0.451387,0.435210,0.505122,0.397838,0.467287,0.479794,0.364651,0.402369
1458,0.476198,0.055537,0.331827,0.148077,0.146701,0.074073,0.265577,0.397670,0.380565,0.105208,...,0.453759,0.456644,0.424670,0.414264,0.497390,0.403660,0.433188,0.525064,0.354776,0.426089


In [206]:
model = DBSCAN(eps=4, min_samples=1).fit(pca_df_normalized)
labels = np.array(list(model.labels_))
np.unique(labels)

array([   0,    1,    2, ..., 1015, 1016, 1017], dtype=int64)

In [207]:
labels[labels == -1].shape

(0,)

In [208]:
labels[labels == 0].shape

(1,)

In [209]:
labels[labels == 1].shape

(443,)

In [210]:
labels[labels == 2].shape

(1,)